In [2]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [3]:
def load_data_from_directories():
    """
    Load image paths and labels from the specified directory structure
    which is one level above the current directory.
    Returns train and validation data for the model.
    """
    import os
    import glob
    
    # Define data directories - one level above the current directory
    base_dir = os.path.join(os.path.dirname(os.getcwd()), 'datasets/brain-tumor')
    train_img_dir = os.path.join(base_dir, 'train/images')
    train_label_dir = os.path.join(base_dir, 'train/labels')
    val_img_dir = os.path.join(base_dir, 'valid/images')
    val_label_dir = os.path.join(base_dir, 'valid/labels')
    
    print(f"Loading data from: {base_dir}")
    
    # Collect all image files
    train_image_paths = sorted(glob.glob(os.path.join(train_img_dir, '*')))
    val_image_paths = sorted(glob.glob(os.path.join(val_img_dir, '*')))
    
    # Get corresponding label files (assuming same filename with different extension)
    train_labels = []
    for img_path in train_image_paths:
        img_filename = os.path.basename(img_path)
        # Remove extension and add label extension (adjust as needed)
        base_name = os.path.splitext(img_filename)[0]
        label_path = os.path.join(train_label_dir, f"{base_name}.txt")
        
        # Read and process label (assuming label files have class as first value)
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                # Parse the first value as class label
                # This assumes YOLO format where first value is class
                line = f.readline().strip()
                if line:
                    parts = line.split()
                    class_id = int(parts[0])  # Extract class ID
                    train_labels.append(class_id)
                else:
                    # Handle empty label file
                    train_labels.append(0)  # Default to class 0 or whatever is appropriate
        else:
            # Handle missing label file
            print(f"Warning: No label found for {img_path}")
            train_labels.append(0)  # Default to class 0 or whatever is appropriate
    
    # Repeat for validation labels
    val_labels = []
    for img_path in val_image_paths:
        img_filename = os.path.basename(img_path)
        base_name = os.path.splitext(img_filename)[0]
        label_path = os.path.join(val_label_dir, f"{base_name}.txt")
        
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                line = f.readline().strip()
                if line:
                    parts = line.split()
                    class_id = int(parts[0])
                    val_labels.append(class_id)
                else:
                    val_labels.append(0)
        else:
            print(f"Warning: No label found for {img_path}")
            val_labels.append(0)
    
    print(f"Loaded {len(train_image_paths)} training images and {len(val_image_paths)} validation images")
    
    return train_image_paths, train_labels, val_image_paths, val_labels

In [4]:
# Custom Dataset class
class BrainTumorDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
            
        return image, torch.tensor(label, dtype=torch.float32)

In [6]:
import torch
import torch.nn as nn
from torchvision.models import vgg16, VGG16_Weights

class VGG16Transfer(nn.Module):
    def __init__(self):
        super(VGG16Transfer, self).__init__()
        
        # Load pretrained VGG16 model with IMAGENET1K_V1 weights
        self.vgg16 = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)
        
        # Freeze all layers in the VGG16 model
        for param in self.vgg16.parameters():
            param.requires_grad = False
            
        # Replace the classifier with our custom classifier
        self.vgg16.classifier = nn.Sequential(
            nn.Linear(25088, 256),  # VGG16's last conv layer outputs 25088 features
            nn.ReLU(),
            nn.Dropout(0.3),        # Dropout after dense layer and ReLU
            nn.Linear(256, 1),      # Binary classification (1 output neuron)
            nn.Sigmoid()            # Sigmoid activation for binary classification
        )
        
        # Unfreeze only the new dense layer (256 units)
        for param in self.vgg16.classifier[0].parameters():  # First Linear layer (256 units)
            param.requires_grad = True
            
    def forward(self, x):
        return self.vgg16(x)

# Get the transforms for preprocessing
transforms = VGG16_Weights.IMAGENET1K_FEATURES.transforms()

# Create model instance
model = VGG16Transfer()

# Print model summary
print(model)

# Verify which layers are trainable
print("\nTrainable parameters:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name}: {param.shape}")

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\TPS-Sean/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100%|██████████| 528M/528M [00:13<00:00, 40.5MB/s] 


VGG16Transfer(
  (vgg16): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      

In [10]:
# Load data
train_image_paths, train_labels, val_image_paths, val_labels = load_data_from_directories()

# Create datasets
train_dataset = BrainTumorDataset(train_image_paths, train_labels, transform=transforms)
val_dataset = BrainTumorDataset(val_image_paths, val_labels, transform=transforms)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Initialize model
model = VGG16Transfer()
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

Loading data from: E:\SDS-CP024-neurovision\submissions-team\andy-chen\model_training\datasets/brain-tumor
Loaded 878 training images and 223 validation images


In [11]:
# Training loop
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20):
    best_val_acc = 0.0
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for inputs, labels in train_loader:
            inputs = inputs.to('cuda' if torch.cuda.is_available() else 'cpu')
            labels = labels.to('cuda' if torch.cuda.is_available() else 'cpu')
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels.view(-1, 1))
            
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            predicted = (outputs > 0.5).float()
            train_total += labels.size(0)
            train_correct += (predicted == labels.view(-1, 1)).sum().item()
        
        train_loss = train_loss / len(train_loader)
        train_acc = 100 * train_correct / train_total
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to('cuda' if torch.cuda.is_available() else 'cpu')
                labels = labels.to('cuda' if torch.cuda.is_available() else 'cpu')
                
                outputs = model(inputs)
                loss = criterion(outputs, labels.view(-1, 1))
                
                val_loss += loss.item()
                predicted = (outputs > 0.5).float()
                val_total += labels.size(0)
                val_correct += (predicted == labels.view(-1, 1)).sum().item()
        
        val_loss = val_loss / len(val_loader)
        val_acc = 100 * val_correct / val_total
        
        # Print statistics
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {train_loss:.4f} - Train Acc: {train_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f} - Val Acc: {val_acc:.2f}%')
        print('-' * 50)
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')



In [12]:
# Start training
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)

Epoch 1/20:
Train Loss: 39.3865 - Train Acc: 48.41%
Val Loss: 27.9017 - Val Acc: 63.68%
--------------------------------------------------
Epoch 2/20:
Train Loss: 35.6404 - Train Acc: 47.72%
Val Loss: 26.5078 - Val Acc: 63.68%
--------------------------------------------------
Epoch 3/20:
Train Loss: 34.9320 - Train Acc: 47.72%
Val Loss: 26.4161 - Val Acc: 63.68%
--------------------------------------------------
Epoch 4/20:
Train Loss: 34.0578 - Train Acc: 47.72%
Val Loss: 26.2093 - Val Acc: 63.68%
--------------------------------------------------
Epoch 5/20:
Train Loss: 33.9660 - Train Acc: 47.72%
Val Loss: 22.9951 - Val Acc: 63.68%
--------------------------------------------------
Epoch 6/20:
Train Loss: 32.3692 - Train Acc: 51.14%
Val Loss: 2.1443 - Val Acc: 63.68%
--------------------------------------------------
Epoch 7/20:
Train Loss: 6.3645 - Train Acc: 57.29%
Val Loss: 0.8238 - Val Acc: 38.57%
--------------------------------------------------
Epoch 8/20:
Train Loss: 0.6296